In [1]:
import newsbank_scraper as ns

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
%pdb off

Automatic pdb calling has been turned OFF


In [4]:
import requests
import pandas
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import re
import urllib
import math


In [5]:
# def  get_all_news(language='English', year='2017', country='USA', sourcetype='Newspaper', 
#                 searchterm='climate%20change', limit=None):
#     # TODO: implement a limit of results in order not to get all of them if we don't want to

results_per_page = 50
# Requesting initial cookie, initializing session
header = {
    'Upgrade-Insecure-Requests' : '1',
    'User-Agent' :'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Mobile Safari/537.36'
}
session = requests.Session()
session.headers.update(header)
mainPage = 'http://infoweb.newsbank.com.proxy.uchicago.edu/resources/?p=AWNB'
response = session.get(mainPage)

# Initial search, page 0
searchURL = get_search_query_url(language=language, year=year, country=country, sourcetype=sourcetype,
                                searchterm=searchterm, page=0)
searchHeader = {
'Host' : 'infoweb.newsbank.com.proxy.uchicago.edu',
'Referer' : 'http://infoweb.newsbank.com.proxy.uchicago.edu/resources/search/nb?p=AWNB&b=results&action=search&fld0=alltext&val0={}&bln1=AND&fld1=YMD_date&val1=&sort=YMD_date%3AD'.format(searchterm),
'Connection' : 'keep-alive',
'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}
response = session.get(searchURL, headers = searchHeader)

max_results = get_max_results(response.text)

all_news = get_news_from_searchpage(response.text, session, country)

if limit:
    max_pages = math.ceil(limit/results_per_page)
else:
    max_pages = math.ceil(max_results/results_per_page)

for page in range(1,max_pages):
    logging.info('New page')
    logging.debug('Current newscount: {}'.format(len(all_news)))
    response = session.get(get_search_query_url(language=language, year=year, country=country,
                                                sourcetype=sourcetype, searchterm=searchterm, page=page), 
                           headers = searchHeader)
    all_news = pandas.concat([get_news_from_searchpage(response.text, session, country), all_news], ignore_index=True)
    
#     return all_news

2018-01-21 16:30:45,715 - DEBUG - Starting new HTTP connection (1): infoweb.newsbank.com.proxy.uchicago.edu
2018-01-21 16:30:45,779 - DEBUG - http://infoweb.newsbank.com.proxy.uchicago.edu:80 "GET /resources/?p=AWNB HTTP/1.1" 302 None
2018-01-21 16:30:45,789 - DEBUG - Starting new HTTPS connection (1): login.proxy.uchicago.edu
2018-01-21 16:30:45,958 - DEBUG - https://login.proxy.uchicago.edu:443 "GET /login?url=http://infoweb.newsbank.com/resources/?p=AWNB HTTP/1.1" 302 None
2018-01-21 16:30:46,061 - DEBUG - Resetting dropped connection: login.proxy.uchicago.edu
2018-01-21 16:30:46,313 - DEBUG - https://login.proxy.uchicago.edu:443 "GET /connect?session=sJjQ6wY3Sk1TqRHY&url=http://infoweb.newsbank.com/resources/?p=AWNB HTTP/1.1" 302 None
2018-01-21 16:30:46,363 - DEBUG - Resetting dropped connection: infoweb.newsbank.com.proxy.uchicago.edu
2018-01-21 16:30:46,487 - DEBUG - http://infoweb.newsbank.com.proxy.uchicago.edu:80 "GET /resources/?p=AWNB HTTP/1.1" 302 None
2018-01-21 16:30:46,

NameError: name 'get_search_query_url' is not defined

In [6]:
response

<Response [200]>

In [3]:
thing = ns.get_all_news()

2018-01-21 16:26:07,873 - DEBUG - Starting new HTTP connection (1): infoweb.newsbank.com.proxy.uchicago.edu
2018-01-21 16:26:07,938 - DEBUG - http://infoweb.newsbank.com.proxy.uchicago.edu:80 "GET /resources/?p=AWNB HTTP/1.1" 302 None
2018-01-21 16:26:07,951 - DEBUG - Starting new HTTPS connection (1): login.proxy.uchicago.edu
2018-01-21 16:26:08,254 - DEBUG - https://login.proxy.uchicago.edu:443 "GET /login?url=http://infoweb.newsbank.com/resources/?p=AWNB HTTP/1.1" 302 None
2018-01-21 16:26:08,354 - DEBUG - Resetting dropped connection: login.proxy.uchicago.edu
2018-01-21 16:26:08,574 - DEBUG - https://login.proxy.uchicago.edu:443 "GET /connect?session=suP1U71yKqmrJMXS&url=http://infoweb.newsbank.com/resources/?p=AWNB HTTP/1.1" 302 None
2018-01-21 16:26:08,614 - DEBUG - Resetting dropped connection: infoweb.newsbank.com.proxy.uchicago.edu
2018-01-21 16:26:08,700 - DEBUG - http://infoweb.newsbank.com.proxy.uchicago.edu:80 "GET /resources/?p=AWNB HTTP/1.1" 302 None
2018-01-21 16:26:08,

LookupError: No result count div found!

In [3]:
thing

,author,date,section,source,text,title
0,Gregory J. Carbone University of South Carolina,"October 3, 2017",Columnists,"Columbus Telegram, The (NE)",(The Conversation is an independent and nonpro...,Why people around the world fear climate\nchan...
1,67 256 235 268BY STUART LEAVENWORTH sleavenwor...,"November 4, 2017",Extra Extra,"State, The (Columbia, SC)",WASHINGTON The Trump administration released a...,"Climate\nchange sucks moisture from the West, ..."
2,Gregory J. Carbone,"October 10, 2017",Houston,Houston Chronicle (TX),>> How the Houston Chronicle covered Harvey. ...,GRAY MATTERS; Why don't Americans fear climate...
3,The Conversation; \nGregory J. Carbone,"October 8, 2017",Final,Patriot,When asked about major threats to their countr...,Other nations fear climate\nchange more
4,Letters,"September 27, 2017",d128e8efdc4258eaca024b869448665e6e1d1,Progress,As world leaders gather at the United Nations ...,Climate\nchange and conflict
5,News,"September 22, 2017",4,"Ridgway Record, The (PA)",As world leaders gather at the United Nations ...,Climate\nChange and Conflict
6,News,"September 22, 2017",4,"Daily Press, The (St. Marys, PA)",As world leaders gather at the United Nations ...,Climate\nChange and Conflict
7,News,"September 22, 2017",4,"Kane Republican, The (PA)",As world leaders gather at the United Nations ...,Climate\nChange and Conflict
8,Rory J. Respicio,"September 20, 2017",Forum,"Guam Daily Post, The (Guam)",President Trump's reversal of U.S. policy on c...,Rising seas and rising costs
9,Junjie Zhang,"July 28, 2017",News,China Daily US Edition (USA),China is already experiencing the effects of c...,Mitigating climate threat to China's food secu...


In [5]:
thing.iloc[0].title

'Why people around the world fear climate\nchange more than Americans do'